In [1]:
## 下面代码来之：https://www.cnblogs.com/always-fight/p/10159547.html
## 

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('.\sohudata\data\sohu_train.txt',sep='\t', header=None)
train_df.head()

,0,1
0,娱乐,《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...
1,娱乐,６．１６日剧榜 ＜最后的朋友＞ 亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...
2,娱乐,超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...
3,娱乐,吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...
4,娱乐,组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...


In [3]:
#train_df  = train_df.dropna()
#train_data = train_df.content.values.tolist()  # 长度是24000

In [4]:
## 查看train 每个分类的名字和样本数量：
for name,group in train_df.groupby(0):
    print(name,"\t",len(group))

体育 	 2000
健康 	 2000
女人 	 2000
娱乐 	 2000
房地产 	 2000
教育 	 2000
文化 	 2000
新闻 	 2000
旅游 	 2000
汽车 	 2000
科技 	 2000
财经 	 2000


In [5]:
### 同样的方法查看test 集合上各个分类名字和样本数量:
test_df = pd.read_csv('.\sohudata\data\sohu_test.txt', sep='\t', header=None)
for name,group in test_df.groupby(0):
    print(name,"\t",len(group))

体育 	 1000
健康 	 1000
女人 	 1000
娱乐 	 1000
房地产 	 1000
教育 	 1000
文化 	 1000
新闻 	 1000
旅游 	 1000
汽车 	 1000
科技 	 1000
财经 	 1000


In [6]:
for name,group in test_df.groupby(0):
    pass
    #print(name[0])
    #print(group[1])

### 1.分词处理

In [4]:
import jieba, time
train_df.columns = ['分类', '文章']
#stopword_list = [k.strip() for k in open('stopwords.txt', encoding='utf-8').readlines() if k.strip() != '']
#上面的语句不建议这么写，因为readlines()是一下子将所有内容读入内存，如果文件过大，会很耗内存，建议这么写
stopword_list = [k.strip() for k in open('.\sohudata\data\stopwords.txt', encoding='utf-8') if k.strip() != '']
cutWords_list = []
 
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' % (i, time.time() - startTime))
    cutWords_list.append(cutWords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\THEONE~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.609 seconds.
Prefix dict has been built succesfully.


前1000篇文章分词共花费52.91秒
前2000篇文章分词共花费102.47秒
前3000篇文章分词共花费207.17秒
前4000篇文章分词共花费307.23秒
前5000篇文章分词共花费401.17秒
前6000篇文章分词共花费499.24秒
前7000篇文章分词共花费526.00秒
前8000篇文章分词共花费552.98秒
前9000篇文章分词共花费594.73秒
前10000篇文章分词共花费634.51秒
前11000篇文章分词共花费680.76秒
前12000篇文章分词共花费731.16秒
前13000篇文章分词共花费761.25秒
前14000篇文章分词共花费789.65秒
前15000篇文章分词共花费828.72秒
前16000篇文章分词共花费869.18秒
前17000篇文章分词共花费908.40秒
前18000篇文章分词共花费945.69秒
前19000篇文章分词共花费1033.60秒
前20000篇文章分词共花费1122.37秒
前21000篇文章分词共花费1144.25秒
前22000篇文章分词共花费1166.31秒
前23000篇文章分词共花费1200.72秒
前24000篇文章分词共花费1234.73秒


In [5]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list.txt','w',encoding = 'utf-8') as file:
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [8]:
# 载入分词文件：
cutWords_list = []
with open('./sohudata/data/cutWords_list.txt','r',encoding = 'utf-8') as file:
    cutWords_list  = [k.split() for k in file]

In [12]:
print("%0.2f"%4)

4.00


### 2.导入word2vec模型

In [12]:
import time
import warnings 
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec
start = time.time()
word2vec_model  = Word2Vec(cutWords_list,size = 100,window = 2,iter = 200,min_count = 20)
end = time.time()
print("训练%d个样本花费的时间是 %0.2f"%(len(cutWords_list),(end - start)/60),'mins')
# sentences 可以是一个list,建议使用BrownCorpus，Text8Corpus或lineSentence构建
# size: 特征向量的维度，默认是100,大的size需要等多的训练数据，但是效果会更好，推荐几十到几百
# min_count: 可以对字典进行截断,词频少于min_count次数的单词会被丢掉，默认值是5
# iter :表示的是迭代次数
# sg: 表示的是训练算法；sg = 0，表示的是CBOW, sg =1，表示的是skim-gram
#alpha： 是学习率的初始化值
#min_count: 如果单词出现的频率小于这个数的时候，就会忽略
#hs : 等于0 的话通过负例采样来模型训练，hs = 1 表示的是通过混层softmax来模型训练
#window：表示当前单词与预测单词之间的最大距离

训练24000个样本花费的时间是 52.07 mins


In [13]:
### 通过Word2Vec 来找跟目标词相近的词，可以返回需要找到几个相近的词，及相似度
word2vec_model.wv.most_similar("开心",topn =10) # 跟词语"运动"相似的4个词分别是：返回的是一个列表，列表里面的
# 元素是元祖(第一个是相关的词汇，第2个是相似度)

[('高兴', 0.6634989976882935),
 ('自豪', 0.6148637533187866),
 ('欣慰', 0.5899534225463867),
 ('快乐', 0.5826623439788818),
 ('愉快', 0.5618224143981934),
 ('生气', 0.5613652467727661),
 ('感动', 0.5603732466697693),
 ('心情', 0.5381132364273071),
 ('对不起', 0.5371996164321899),
 ('兴奋', 0.523831307888031)]

In [4]:
### word2vec_model 方法使用postive 和negative 这2个关键词参数的简单实例，“女人 - ？ = 男人 - 先生
word2vec_model.most_similar(positive = ['女人','先生'],negative = ['男人'],topn = 1)

[('女士', 0.7172855138778687)]

In [14]:
# 保存模型
word2vec_model.save('./sohudata/data/word2vec_model.w2v')

### 3.1重新加载模型

In [12]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec.load('./sohudata/data/word2vec_model.w2v')
word2vec_model.wv.most_similar('开心',topn= 4)

[('高兴', 0.6634989976882935),
 ('自豪', 0.6148637533187866),
 ('欣慰', 0.5899534225463867),
 ('快乐', 0.5826623439788818)]

### 4 特征工程

In [ ]:
### 对于每一篇文章，获取文章的每个分词在word2vec模型中的向量；然后每一篇文章中词分词都可以在word2vec模型中的
### 相关性向量的求和后的平均数，即此篇文章在word2vec模型中的相关向量
### 下面实例化第一个Word2Vec一个对象时，关键字参数size = 100,相关性矩阵都是100
### 新建一个getVector 计算一篇文章的词向量，传入2个参数，1个是word分词结果，1个是word2vec 模型对象

In [13]:
def getVector_v1(cutWords,word2vec_model):
    count = 0
    article_vector = np.zeros(word2vec_model.layer1_size)
    for cutWord in cutWords:
        if cutWord in word2vec_model:
            article_vector+= word2vec_model[cutWord]
            count+=1 
    return article_vector/count # 返回的是文档中每个词对应词向量的平均值

In [ ]:
# 取一部分数量的词来更换成词向量
import time
import numpy as np
cutword_nums = 24000
start = time.time()
vector_list = []
i = 0
for cutWords in cutWords_list[:cutword_nums]:
    vector_list.append(getVector_v1(cutWords,word2vec_model))
    i+=1
    if i%1000 == 0:
        print("前%d篇文章所形成词向量花费的时间%0.2f" %(i,time.time() - start),'sec')
X = np.array(vector_list)
print("处理全部花费的时间:%0.2f"%(time.time() - start),'sec')

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  app.launch_new_instance()
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


前1000篇文章所形成词向量花费的时间11.08 sec
前2000篇文章所形成词向量花费的时间21.79 sec


In [29]:
X[0],len(X[0])

(array([-0.6957985 ,  0.53137462,  1.04064428, -0.2385629 ,  0.12440227,
         0.16879624, -0.14543796,  0.05181002, -0.18992081,  0.32969225,
         0.09313782,  0.25629261,  0.1265765 , -0.66114717, -0.325841  ,
        -0.13366155, -1.12290873, -0.00698046, -0.09335969,  0.43577012,
         0.55911621, -1.06186359, -0.1593309 ,  0.48744267,  0.19711087,
        -0.43444411, -0.72344278,  0.29048578, -0.21027578, -0.07362615,
         0.49724962,  0.1162595 ,  0.04271776, -0.42054305, -0.33989077,
         0.89218481, -0.27475017,  0.10951941, -0.24385046,  0.05179802,
         0.8296283 ,  0.1879013 , -0.38489069, -0.35859662,  0.17663303,
         0.58216028, -0.05308534, -0.30152697,  0.74189372,  0.60776241,
         0.52278352,  0.2623954 , -0.57246163,  0.67103611,  0.27365632,
        -0.34883797,  1.00951477,  0.74290665, -0.42941052,  0.64676393,
         0.88888832,  0.18519186, -0.11065338, -0.0743731 , -0.42954011,
         0.70717746, -0.24690223, -0.01141937,  0.6

In [24]:
### j将计算的特征矩阵save成文档来
X.dump('./sohudata/data/articles_vector.txt') # dump方法传入一个字符串

AttributeError: dump not found

In [60]:
import numpy as np
X = np.load('./sohudata/data/articles_vector.txt',allow_pickle=True)

### 5.模型训练及模型评估

In [1]:
### 1）标签编码成为数字
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train_df         = pd.read_csv('./sohudata/data/sohu_train.txt', sep='\t', header=None)
train_df.columns = ['label','example']
labelencoder = LabelEncoder()
y            = labelencoder.fit_transform(train_df['label'])

In [4]:
y[22],train_df['label'][22]

(3, '娱乐')

In [40]:
### 2)引入LR回归模型
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
train_X,text_X,train_y,test_y = train_test_split(X,y,test_size = 0.2,random_state = 200)
logistic_model = LogisticRegression()
logistic_model.fit(train_X,train_y)
print("准确率：%0.2f"%(logistic_model.score(text_X,test_y)*100),"%")

准确率：77.12 %


In [41]:
### 通过joblib来保存model
from sklearn.externals import joblib
joblib.dump(logistic_model,'./sohudata/data/logistic.model')

In [1]:
## 加载model
from sklearn.externals import joblib
logistic_model = joblib.load('./sohudata/data/logistic.model')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


FileNotFoundError: [Errno 2] No such file or directory: './sohudata/data/logistic.model'

In [44]:
### 交叉验证
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
cv_split = ShuffleSplit(len(X),train_size = 0.7,test_size = 0.2)
log_model       = LogisticRegression()
score_ndarray   = cross_val_score(log_model,X,y,cv = cv_split)
print(score_ndarray)
print(score_ndarray.mean())

[0.78083333 0.78145833 0.78770833 0.791875   0.77875    0.77770833
 0.78354167 0.78041667 0.791875   0.784375  ]
0.7838541666666667


In [67]:
### m模型测试
import jieba
def getVectorMatrix(article_series):
    return np.array([getVector_v1(jieba.cut(k),word2vec_model) for k in article_series])
test_df = pd.read_csv('./sohudata/data/sohu_test.txt', sep='\t', header=None)
logistic_model = joblib.load('./sohudata/data/logistic.model')
test_df.columns = ['分类','文章']
for name,group in test_df.groupby('分类'):
    featurematrix = getVectorMatrix(group['文章'])
    target        = labelencoder.transform(group['分类'])
    print(name,logistic_model.score(featurematrix,target))

C:\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: The file './sohudata/data/logistic.model' has been generated with a joblib version less than 0.10. Please regenerate this pickle file.


NameError: name 'getVector_v1' is not defined

In [64]:
### 计算准确率和召回率
from sklearn.metrics import classification_report
test_df = pd.read_csv('./sohudata/data/sohu_test.txt', sep='\t', header=None)
test_df.columns = ['分类','文章']
test_label      = labelencoder.transform(test_df['分类'])
test_pred       = logistic_model.score(getVectorMatrix(test_df['文章']),test_label)
print(labelencoder.inverse_transform([[x] for x in range(12)]))
print(classification_report(test_label,test_pred))

NameError: name 'getVectorMatrix' is not defined

In [ ]:
##项目分割线###--------------------------------------------###项目分割线

## 第2大部分，通过TFidfVectorize 模型来抽取特征完成完成文本分类

In [3]:
cutWords_list = []
with open('./sohudata/data/cutWords_list.txt','r',encoding = 'utf-8') as file:
    cutWords_list = [k.split() for k in file] 

In [5]:
###1.TFidfVectorize 模型 - 通过sklearn.feature_extraction import tfidfVectorize 实现文本特征的抽取
stopword_list = [k.strip() for k in open('.\sohudata\data\stopwords.txt', encoding='utf-8') if k.strip() != '']
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(cutWords_list,stop_words =stopword_list,min_df = 40,max_df = 0.3)

In [40]:
### 特征工程
import pandas as pd
train_df = pd.read_csv('./sohudata/data/sohu_train.txt', sep='\t', header=None)
train_df.columns = ['分类','文章']
X = tfidf.fit_transform(train_df['文章'])
print("词表大小:",len(tfidf.vocabulary_))
print(X.shape)  # 可见每篇文章内容被向量化，维度特征值是: 3946

词表大小: 3946
(24000, 3946)


In [41]:
### 训练数据 - 标签 
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train_df = pd.read_csv('.\sohudata\data\sohu_train.txt',sep='\t', header=None)
train_df.head()
import pandas as pd
y        = labelencoder.fit_transform(train_df[0])
y.shape

(24000,)

In [42]:
### 逻辑回归
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
train_X,text_X,train_y,test_y = train_test_split(X,y,test_size = 0.2,random_state = 200)
log_clf = LogisticRegression(multi_class = 'multinomial',solver = 'lbfgs')
log_clf.fit(train_X,train_y)
print("准确率是%0.2f"%(100*log_clf.score(text_X,test_y)),'%')

准确率是86.71 %


In [43]:
#save 模型
import pickle
file = open('./sohudata/data/tfidf.model','wb')
save_data = {
    'labelencoder':labelencoder,
    'tfidf':tfidf,
    'log_clf':log_clf
}
pickle.dump(save_data,file)

In [46]:
# 重新加载模型
import pickle
with open('./sohudata/data/tfidf.model', 'rb') as file:  #读取2进制文件
    tfidf_model = pickle.load(file)
    tfidfVectorizer = tfidf_model['tfidf']
    labelEncoder = tfidf_model['labelencoder']
    logistic_model = tfidf_model['log_clf']

In [47]:
# 重新获得特征和标签值
import pandas as pd
train_df = pd.read_csv('./sohudata/data/sohu_train.txt', sep='\t', header=None)
X = tfidfVectorizer.transform(train_df[1])
y = labelEncoder.transform(train_df[0])

In [52]:
X.shape[0]

24000

In [53]:
### 交叉验证
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
cv_split        = ShuffleSplit(X.shape[0],train_size = 0.7,test_size = 0.2) # X.shape[0] = 24000
log_model       = LogisticRegression(multi_class='multinomial', solver='lbfgs')
score_ndarray   = cross_val_score(log_model,X,y,cv = cv_split)
print(score_ndarray)
print("交叉验证的准确率是%0.2f"%(score_ndarray.mean()*100),"%")

[ 0.875       0.87625     0.87458333  0.8775      0.87416667  0.87958333
  0.87416667  0.86625     0.87354167  0.86958333]
交叉验证的准确率是87.41 %


In [54]:
##模型评估 - 混淆矩阵
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
import pandas as pd
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logistic_model = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
predict_y = logistic_model.predict(test_X)
pd.DataFrame(confusion_matrix(test_y, predict_y),columns=labelEncoder.classes_, index=labelEncoder.classes_)

,体育,健康,女人,娱乐,房地产,教育,文化,新闻,旅游,汽车,科技,财经
体育,405,0,0,2,5,2,6,6,0,2,4,2
健康,0,384,5,2,2,1,2,3,2,3,9,2
女人,2,3,349,5,3,2,4,1,1,3,0,2
娱乐,5,1,2,361,1,1,5,2,2,2,1,0
房地产,2,3,0,2,365,2,11,21,3,2,6,10
教育,2,4,3,2,4,357,2,2,2,5,2,3
文化,5,2,6,10,6,3,338,6,4,3,4,0
新闻,4,1,1,2,12,3,5,331,2,2,5,9
旅游,3,2,2,4,5,4,8,8,361,2,3,3
汽车,3,5,1,2,5,8,6,2,5,340,2,5


In [57]:
### 绘制precision、recall、f1-score、support报告表：
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
 
def eval_model(y_true, y_pred, labels):
    #计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support( y_true, y_pred)
    #计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision' : p,
        u'Recall' : r,
        u'F1' : f1,
        u'Support' : s
    })
     
    res2 = pd.DataFrame({
        u'Label' : ['总体'],
        u'Precision' : [tot_p],
        u'Recall': [tot_r],
        u'F1' : [tot_f1],
        u'Support' : [tot_s]
    })
     
    res2.index = [12]
    res = pd.concat( [res1, res2])
    return res[ ['Label', 'Precision', 'Recall', 'F1', 'Support'] ]
 
predict_y = logistic_model.predict(test_X)
eval_model(test_y, predict_y, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.959077,0.914000,0.935996,1000
1,健康,0.919802,0.929000,0.924378,1000
2,女人,0.936874,0.935000,0.935936,1000
3,娱乐,0.925595,0.933000,0.929283,1000
4,房地产,0.831361,0.843000,0.837140,1000
5,教育,0.912475,0.907000,0.909729,1000
6,文化,0.841193,0.874000,0.857283,1000
7,新闻,0.814643,0.879000,0.845599,1000
8,旅游,0.941922,0.892000,0.916281,1000
9,汽车,0.920945,0.897000,0.908815,1000


In [58]:
### 模型测试
import pandas as pd

test_df = pd.read_csv('./sohudata/data/sohu_test.txt', sep='\t', header=None)
test_X = tfidfVectorizer.transform(test_df[1])
test_y = labelEncoder.transform(test_df[0])
predict_y = logistic_model.predict(test_X)

eval_model(test_y, predict_y, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,体育,0.959077,0.914000,0.935996,1000
1,健康,0.919802,0.929000,0.924378,1000
2,女人,0.936874,0.935000,0.935936,1000
3,娱乐,0.925595,0.933000,0.929283,1000
4,房地产,0.831361,0.843000,0.837140,1000
5,教育,0.912475,0.907000,0.909729,1000
6,文化,0.841193,0.874000,0.857283,1000
7,新闻,0.814643,0.879000,0.845599,1000
8,旅游,0.941922,0.892000,0.916281,1000
9,汽车,0.920945,0.897000,0.908815,1000
